In [303]:
!pip install optuna

In [304]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn import svm
import optuna
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [305]:
# getting data from source
df_genomicMatrix=pd.read_csv('https://ucsc-public-main-xena-hub.s3.us-east-1.amazonaws.com/download/miller2005_public%2Fmiller2005_genomicMatrix.gz',delimiter='\t',header=None)
df_clinicalMatrix=pd.read_csv('https://ucsc-public-main-xena-hub.s3.us-east-1.amazonaws.com/download/miller2005_public%2Fmiller2005_public_clinicalMatrix.gz',delimiter='\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251) ha

In [306]:
# first few rows of clinical matrix
df_clinicalMatrix.head(10)

,sampleID,AgeAtDiagnosis,DLDAerror,DSS,DSS.time,ERstatus,EistonHistologicGrade,LymphNodeStatus,PgRstatus,TumorSize,_INTEGRATION,_PATIENT,p53DLDAclassResult,p53mutant,_GENOMIC_ID_miller2005
0,X100B08,68,No,AliveOrCensored,12.0,ER+,G1,LN-,PgR-,9.0,X100B08,X100B08,WT-like,p53-,X100B08
1,X101B88,40,No,AliveOrCensored,12.0,ER-,G3,LN-,PgR-,12.0,X101B88,X101B88,MT-like,p53+,X101B88
2,X102B06,51,No,AliveOrCensored,12.0,ER+,G3,LN-,PgR+,26.0,X102B06,X102B06,MT-like,p53+,X102B06
3,X103B41,52,Yes,NaN,NaN,ER-,G3,LN+,PgR-,22.0,X103B41,X103B41,MT-like,p53-,X103B41
4,X104B91,80,Yes,AliveOrCensored,4.0,ER+,G3,NaN,PgR+,24.0,X104B91,X104B91,WT-like,p53+,X104B91
5,X105B13,46,No,AliveOrCensored,12.0,ER+,G1,LN-,PgR+,13.0,X105B13,X105B13,WT-like,p53-,X105B13
6,X106B55,37,No,DeathFromCancer,7.0,ER+,G1,LN-,PgR+,60.0,X106B55,X106B55,WT-like,p53-,X106B55
7,X10B88,70,No,AliveOrCensored,11.0,ER+,G1,LN-,PgR+,50.0,X10B88,X10B88,WT-like,p53-,X10B88
8,X110B34,74,No,AliveOrCensored,12.0,ER+,G2,LN-,PgR+,20.0,X110B34,X110B34,MT-like,p53+,X110B34
9,X111B51,41,No,DeathFromCancer,7.0,ER+,G3,LN-,PgR+,33.0,X111B51,X111B51,MT-like,p53+,X111B51


In [307]:
df_clinicalMatrix.columns

Index(['sampleID', 'AgeAtDiagnosis', 'DLDAerror', 'DSS', 'DSS.time',
       'ERstatus', 'EistonHistologicGrade', 'LymphNodeStatus', 'PgRstatus',
       'TumorSize', '_INTEGRATION', '_PATIENT', 'p53DLDAclassResult',
       'p53mutant', '_GENOMIC_ID_miller2005'],
      dtype='object')

In [308]:
# first few rows of genomic matrix
df_genomicMatrix.head(10)

,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
0,probe,X100B08,X101B88,X102B06,X103B41,X104B91,X105B13,X106B55,X10B88,X110B34,...,X85A03,X86A40,X87A79,X88A67,X8B87,X90A63,X94A16,X96A21,X99A50,X9B52
1,211050_x_at,-0.60609845997949,-0.71211732736651,-0.24902371657542,0.47339419538929,-0.54907184258026,-0.3092005016715,0.41897323719004,-0.3946572303695,-0.47251557515557,...,0.19425738895826,-0.0147835375762102,0.13407539167209,-0.19990245782577,0.11891965170859,-0.37927029343351,0.82007590513398,-0.0455767367670501,-0.0245526540187702,0.27066630092569
2,220399_at,0.297761884637424,0.108554028539094,0.529915688630775,-0.0236977596184555,-0.240674552924416,-0.186094550508416,0.00280455771169397,-0.131805918000736,0.113294687596224,...,0.158997493199424,0.228584615788685,-1.61824983812591,-0.237390400226426,0.228422630263264,-0.433716710549096,-1.76092467281198,0.290774879921944,0.248706789784914,-0.0227919448987057
3,202115_s_at,-0.0207199586993729,-0.329812842420013,-0.455559869614984,0.335866503578156,0.0914735642808466,-0.0649335936114834,0.232192293522567,-0.0245333551689031,0.390957101123317,...,0.611860013300097,0.303447871810647,0.0735930672214167,-0.082834904637803,0.133630190202147,0.0553951474851369,0.0711540480339563,-0.114437082375583,0.242832779585116,0.218242406811207
4,224501_at,0.0191163830257484,-0.508559867759712,0.0562016961050977,-0.0112785620922615,0.189045651599998,0.696562269810068,0.192981870643838,-0.319990873927932,-0.243858578920162,...,0.209863932679728,0.215091978389048,-0.323448141350642,-0.0203078647708521,0.214314624350358,0.212822716079898,0.368002424465348,0.207985730041758,0.267992659684308,0.791908891893848
5,227347_x_at,0.217003285274825,0.750268555716166,0.469504094265635,-0.182405554283685,-0.472259746631345,0.294003049170574,0.147631658679885,0.462857150607994,0.781625005612315,...,-0.792750352948505,0.744522780941135,0.156746782941595,-0.0604454092963254,0.107038907434855,-0.192823561730845,0.0307527555541549,-0.463568592211125,0.116093520978945,-0.142528684288885
6,205483_s_at,-0.959765619734105,-0.739373617253365,0.436695502974866,0.913890761259296,1.18484495173342,-0.921591248427505,-0.976086516126244,1.81810520319521,-0.723100883245385,...,-1.11561706495908,-1.75963015795501,0.355280087728865,0.881458469512495,-0.714931953437455,0.971320638022425,1.83952061250142,-0.695891246645205,-1.02142827267055,1.35853694226627
7,212283_at,0.31905965380238,-0.63507596067365,0.253222377924609,0.134584773097739,0.204097424289389,-1.48858656175022,0.124965967004189,-0.399400768147571,-1.10388472055312,...,-0.0234020299288709,-0.785423593070711,-0.148167985995971,-0.0653804748145212,0.670274920597399,-0.620541625136491,-0.131310737024201,-0.275498863350731,-0.333262132498781,0.217372783731449
8,212285_s_at,0.26743053282687,0.404148351178069,0.236893737236249,0.46841163348552,0.397235706101899,0.47611631765845,0.405963629525349,0.0941344988476995,0.373420130159879,...,0.0205554728932693,-0.606536417971721,-0.162472454063082,-0.520093286319691,0.127405316667399,-0.30007531133036,-0.134719368061351,-0.61724686479563,-0.00359567394769122,-0.083492616828881
9,217419_x_at,0.0546245393746343,0.553476301226864,0.263843536673264,0.508556696455114,0.107319891585904,0.844849143201444,0.0171650830981847,0.264637970710144,0.493297784508725,...,0.018747293370545,-1.03597918814879,-0.0122102766741659,-0.563183591738616,0.252069009905044,-0.227527908675295,-0.0885185694443251,-0.927063458578016,-0.0206992687985554,-0.166205147103785


In [309]:
# transpose of genomic matrix data 
df_genomicMatrix_t=df_genomicMatrix.T

In [310]:
df_genomicMatrix_t.columns = df_genomicMatrix_t.iloc[0] # converting first row into column names 
df_genomicMatrix_t=df_genomicMatrix_t.iloc[1:] # deleting forst row due to duplicate

In [311]:
df_genomicMatrix_t

,probe,211050_x_at,220399_at,202115_s_at,224501_at,227347_x_at,205483_s_at,212283_at,212285_s_at,217419_x_at,...,216922_x_at,208331_at,211461_at,232402_at,207647_at,207646_s_at,207916_at,202829_s_at,214950_at,208164_s_at
1,X100B08,-0.60609845997949,0.297761884637424,-0.0207199586993729,0.0191163830257484,0.217003285274825,-0.959765619734105,0.31905965380238,0.26743053282687,0.0546245393746343,...,0.585269,0.677886,0.294886,0.334739,-0.922423,0.525822,0.65593,-0.373383,0.533709,0.9551
2,X101B88,-0.71211732736651,0.108554028539094,-0.329812842420013,-0.508559867759712,0.750268555716166,-0.739373617253365,-0.63507596067365,0.404148351178069,0.553476301226864,...,0.34349,-0.37717,-0.146836,-0.027103,0.108189,-0.108456,0.800626,0.047387,-0.630866,0.550008
3,X102B06,-0.24902371657542,0.529915688630775,-0.455559869614984,0.0562016961050977,0.469504094265635,0.436695502974866,0.253222377924609,0.236893737236249,0.263843536673264,...,-1.192292,-0.264924,-0.403073,-0.273291,0.408303,0.256424,0.052404,-0.076766,0.170939,-0.016254
4,X103B41,0.47339419538929,-0.0236977596184555,0.335866503578156,-0.0112785620922615,-0.182405554283685,0.913890761259296,0.134584773097739,0.46841163348552,0.508556696455114,...,0.233813,-0.182134,-0.132613,0.314508,0.147834,-0.766659,-1.517961,0.487872,0.394624,0.440938
5,X104B91,-0.54907184258026,-0.240674552924416,0.0914735642808466,0.189045651599998,-0.472259746631345,1.18484495173342,0.204097424289389,0.397235706101899,0.107319891585904,...,-0.063716,-0.051118,-0.149773,-0.304061,-1.229899,-0.000545,0.574513,0.597032,0.196972,0.120323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,X90A63,-0.37927029343351,-0.433716710549096,0.0553951474851369,0.212822716079898,-0.192823561730845,0.971320638022425,-0.620541625136491,-0.30007531133036,-0.227527908675295,...,-0.206234,-0.513839,-0.401776,-0.095711,-0.100813,-1.245132,-1.202338,0.419989,0.774586,0.810731
248,X94A16,0.82007590513398,-1.76092467281198,0.0711540480339563,0.368002424465348,0.0307527555541549,1.83952061250142,-0.131310737024201,-0.134719368061351,-0.0885185694443251,...,0.15852,-0.533802,0.656851,-0.550142,1.029998,0.569329,-0.975293,-0.121131,0.291052,-0.603854
249,X96A21,-0.0455767367670501,0.290774879921944,-0.114437082375583,0.207985730041758,-0.463568592211125,-0.695891246645205,-0.275498863350731,-0.61724686479563,-0.927063458578016,...,-0.176975,0.228833,0.444697,0.008805,0.853332,-0.181494,0.375052,-0.038691,-0.111655,1.093266
250,X99A50,-0.0245526540187702,0.248706789784914,0.242832779585116,0.267992659684308,0.116093520978945,-1.02142827267055,-0.333262132498781,-0.00359567394769122,-0.0206992687985554,...,-1.34396,0.001666,0.222738,0.291546,0.130782,-1.347966,-0.827747,-0.286596,0.225008,0.760187


In [312]:
df_genomicMatrix_t.rename(columns={'probe': 'sampleID'}, inplace=True) # renaming first column to make a join with the clinicalMatrix dataframe

In [313]:
df_genomicMatrix_t

,sampleID,211050_x_at,220399_at,202115_s_at,224501_at,227347_x_at,205483_s_at,212283_at,212285_s_at,217419_x_at,...,216922_x_at,208331_at,211461_at,232402_at,207647_at,207646_s_at,207916_at,202829_s_at,214950_at,208164_s_at
1,X100B08,-0.60609845997949,0.297761884637424,-0.0207199586993729,0.0191163830257484,0.217003285274825,-0.959765619734105,0.31905965380238,0.26743053282687,0.0546245393746343,...,0.585269,0.677886,0.294886,0.334739,-0.922423,0.525822,0.65593,-0.373383,0.533709,0.9551
2,X101B88,-0.71211732736651,0.108554028539094,-0.329812842420013,-0.508559867759712,0.750268555716166,-0.739373617253365,-0.63507596067365,0.404148351178069,0.553476301226864,...,0.34349,-0.37717,-0.146836,-0.027103,0.108189,-0.108456,0.800626,0.047387,-0.630866,0.550008
3,X102B06,-0.24902371657542,0.529915688630775,-0.455559869614984,0.0562016961050977,0.469504094265635,0.436695502974866,0.253222377924609,0.236893737236249,0.263843536673264,...,-1.192292,-0.264924,-0.403073,-0.273291,0.408303,0.256424,0.052404,-0.076766,0.170939,-0.016254
4,X103B41,0.47339419538929,-0.0236977596184555,0.335866503578156,-0.0112785620922615,-0.182405554283685,0.913890761259296,0.134584773097739,0.46841163348552,0.508556696455114,...,0.233813,-0.182134,-0.132613,0.314508,0.147834,-0.766659,-1.517961,0.487872,0.394624,0.440938
5,X104B91,-0.54907184258026,-0.240674552924416,0.0914735642808466,0.189045651599998,-0.472259746631345,1.18484495173342,0.204097424289389,0.397235706101899,0.107319891585904,...,-0.063716,-0.051118,-0.149773,-0.304061,-1.229899,-0.000545,0.574513,0.597032,0.196972,0.120323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,X90A63,-0.37927029343351,-0.433716710549096,0.0553951474851369,0.212822716079898,-0.192823561730845,0.971320638022425,-0.620541625136491,-0.30007531133036,-0.227527908675295,...,-0.206234,-0.513839,-0.401776,-0.095711,-0.100813,-1.245132,-1.202338,0.419989,0.774586,0.810731
248,X94A16,0.82007590513398,-1.76092467281198,0.0711540480339563,0.368002424465348,0.0307527555541549,1.83952061250142,-0.131310737024201,-0.134719368061351,-0.0885185694443251,...,0.15852,-0.533802,0.656851,-0.550142,1.029998,0.569329,-0.975293,-0.121131,0.291052,-0.603854
249,X96A21,-0.0455767367670501,0.290774879921944,-0.114437082375583,0.207985730041758,-0.463568592211125,-0.695891246645205,-0.275498863350731,-0.61724686479563,-0.927063458578016,...,-0.176975,0.228833,0.444697,0.008805,0.853332,-0.181494,0.375052,-0.038691,-0.111655,1.093266
250,X99A50,-0.0245526540187702,0.248706789784914,0.242832779585116,0.267992659684308,0.116093520978945,-1.02142827267055,-0.333262132498781,-0.00359567394769122,-0.0206992687985554,...,-1.34396,0.001666,0.222738,0.291546,0.130782,-1.347966,-0.827747,-0.286596,0.225008,0.760187


In [314]:
df_genomicMatrix_t.columns

Index(['sampleID', '211050_x_at', '220399_at', '202115_s_at', '224501_at',
       '227347_x_at', '205483_s_at', '212283_at', '212285_s_at', '217419_x_at',
       ...
       '216922_x_at', '208331_at', '211461_at', '232402_at', '207647_at',
       '207646_s_at', '207916_at', '202829_s_at', '214950_at', '208164_s_at'],
      dtype='object', name=0, length=24712)

In [321]:
# joining the two dataframes
df_inner = pd.merge(df_genomicMatrix_t, df_clinicalMatrix, on='sampleID',how='inner')
df_inner.head(5)

,sampleID,211050_x_at,220399_at,202115_s_at,224501_at,227347_x_at,205483_s_at,212283_at,212285_s_at,217419_x_at,...,ERstatus,EistonHistologicGrade,LymphNodeStatus,PgRstatus,TumorSize,_INTEGRATION,_PATIENT,p53DLDAclassResult,p53mutant,_GENOMIC_ID_miller2005
0,X100B08,-0.60609845997949,0.297761884637424,-0.0207199586993729,0.0191163830257484,0.217003285274825,-0.959765619734105,0.31905965380238,0.26743053282687,0.0546245393746343,...,ER+,G1,LN-,PgR-,9.0,X100B08,X100B08,WT-like,p53-,X100B08
1,X101B88,-0.71211732736651,0.108554028539094,-0.329812842420013,-0.508559867759712,0.750268555716166,-0.739373617253365,-0.63507596067365,0.404148351178069,0.553476301226864,...,ER-,G3,LN-,PgR-,12.0,X101B88,X101B88,MT-like,p53+,X101B88
2,X102B06,-0.24902371657542,0.529915688630775,-0.455559869614984,0.0562016961050977,0.469504094265635,0.436695502974866,0.253222377924609,0.236893737236249,0.263843536673264,...,ER+,G3,LN-,PgR+,26.0,X102B06,X102B06,MT-like,p53+,X102B06
3,X103B41,0.47339419538929,-0.0236977596184555,0.335866503578156,-0.0112785620922615,-0.182405554283685,0.913890761259296,0.134584773097739,0.46841163348552,0.508556696455114,...,ER-,G3,LN+,PgR-,22.0,X103B41,X103B41,MT-like,p53-,X103B41
4,X104B91,-0.54907184258026,-0.240674552924416,0.0914735642808466,0.189045651599998,-0.472259746631345,1.18484495173342,0.204097424289389,0.397235706101899,0.107319891585904,...,ER+,G3,NaN,PgR+,24.0,X104B91,X104B91,WT-like,p53+,X104B91


In [328]:
df_inner=df_inner.dropna()

In [329]:
# Dropping other output variables to predict only EistonHistologicGrade
df_inner_1 = df_inner.drop(columns=['AgeAtDiagnosis','DLDAerror', 'DSS', 'DSS.time',
       'ERstatus','LymphNodeStatus', 'PgRstatus',
       'TumorSize', '_INTEGRATION', '_PATIENT', 'p53DLDAclassResult',
       'p53mutant', '_GENOMIC_ID_miller2005'])

In [330]:
print(df_inner_1['EistonHistologicGrade'].value_counts().sort_values(ascending=False))

G2    117
G1     58
G3     47
Name: EistonHistologicGrade, dtype: int64



distributions
correlation
outlier

In [331]:
# Splitting data into train and test
X=df_inner_1.drop(columns=[ 'EistonHistologicGrade','sampleID'],axis=1) 
y=df_inner['EistonHistologicGrade']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [351]:
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

pca = PCA(n_components=100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)


In [352]:
X_train

array([[ 0.28408791, -0.24998374,  0.06577851, ...,  0.13563617,
        -0.00157505,  0.0533327 ],
       [-0.22888611,  0.39739324, -0.1468484 , ...,  0.07900621,
         0.13818102, -0.26393729],
       [ 0.44782284, -0.1262454 ,  0.22516654, ..., -0.11379768,
         0.00893464,  0.13005307],
       ...,
       [ 0.31362842, -0.13763871, -0.3660912 , ..., -0.05126773,
         0.50171663,  0.22787998],
       [-0.17467378, -0.07365739,  0.41972437, ..., -0.13824629,
         0.09406414,  0.20045405],
       [-0.2099614 , -0.10345822,  0.15228527, ..., -0.46628064,
        -0.06222487,  0.17229564]])

In [353]:
total_var = pca.explained_variance_ratio_.sum() * 100
total_var

99.99999999999997

In [354]:
X_train = pd.DataFrame(data = X_train)
X_test = pd.DataFrame(data = X_test)

In [283]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Columns: 168 entries, 0 to 167
dtypes: float64(168)
memory usage: 220.6 KB


In [294]:
X_train.isnull().sum()

0      0
1      0
2      0
3      0
4      0
      ..
163    0
164    0
165    0
166    0
167    0
Length: 168, dtype: int64

In [287]:
X_test.isnull().sum()

0      0
1      0
2      0
3      0
4      0
      ..
163    0
164    0
165    0
166    0
167    0
Length: 168, dtype: int64

In [285]:
X_train.duplicated().sum()

0

In [286]:
X_test.duplicated().sum()

0

In [356]:
modelsvm=svm.SVC()
modelsvm.fit(X_train, y_train)

SVC()

In [357]:
pred=modelsvm.predict(X_test)
accuracy=accuracy_score(y_test, pred)
print('Accuracy:' ,accuracy)

Accuracy: 0.5540540540540541


In [358]:
def objective(trial):
    params = {
        'kernel':trial.suggest_categorical('kernel',['rbf','poly','linear','sigmoid']),
        'C': trial.suggest_float("C",0.1,3.0,log=True),
        'gamma': trial.suggest_categorical('gamma',['auto','scale']),
        'degree': trial.suggest_int("degree",1,3,log=True),
    }
    modelsvm2 =svm.SVC(**params)
    modelsvm2.fit(X_train, y_train)
    preds = modelsvm2.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy
study = optuna.create_study(direction="maximize")
print(f"Sampler is {study.sampler.__class__.__name__}")
study.optimize(objective, n_trials=100, timeout=100)

[I 2022-03-09 17:01:55,953] A new study created in memory with name: no-name-c9ba6845-46a7-4800-9875-6ed10d0fdb2f
[I 2022-03-09 17:01:55,967] Trial 0 finished with value: 0.5675675675675675 and parameters: {'kernel': 'poly', 'C': 0.2480354796593595, 'gamma': 'auto', 'degree': 2}. Best is trial 0 with value: 0.5675675675675675.
[I 2022-03-09 17:01:55,983] Trial 1 finished with value: 0.5675675675675675 and parameters: {'kernel': 'rbf', 'C': 0.5159988457051253, 'gamma': 'auto', 'degree': 3}. Best is trial 0 with value: 0.5675675675675675.
[I 2022-03-09 17:01:55,997] Trial 2 finished with value: 0.5405405405405406 and parameters: {'kernel': 'linear', 'C': 0.7444713031510276, 'gamma': 'auto', 'degree': 1}. Best is trial 0 with value: 0.5675675675675675.
[I 2022-03-09 17:01:56,011] Trial 3 finished with value: 0.4864864864864865 and parameters: {'kernel': 'linear', 'C': 1.9993233126539702, 'gamma': 'scale', 'degree': 1}. Best is trial 0 with value: 0.5675675675675675.
[I 2022-03-09 17:01:56

Sampler is TPESampler


[I 2022-03-09 17:01:56,164] Trial 12 finished with value: 0.5675675675675675 and parameters: {'kernel': 'rbf', 'C': 0.725107545016592, 'gamma': 'auto', 'degree': 2}. Best is trial 0 with value: 0.5675675675675675.
[I 2022-03-09 17:01:56,192] Trial 13 finished with value: 0.5675675675675675 and parameters: {'kernel': 'sigmoid', 'C': 0.28029737917211195, 'gamma': 'auto', 'degree': 3}. Best is trial 0 with value: 0.5675675675675675.
[I 2022-03-09 17:01:56,214] Trial 14 finished with value: 0.5675675675675675 and parameters: {'kernel': 'rbf', 'C': 1.1516589230258707, 'gamma': 'auto', 'degree': 2}. Best is trial 0 with value: 0.5675675675675675.
[I 2022-03-09 17:01:56,231] Trial 15 finished with value: 0.5675675675675675 and parameters: {'kernel': 'poly', 'C': 0.2690128067933237, 'gamma': 'scale', 'degree': 2}. Best is trial 0 with value: 0.5675675675675675.
[I 2022-03-09 17:01:56,253] Trial 16 finished with value: 0.5675675675675675 and parameters: {'kernel': 'poly', 'C': 2.820398761260088

In [359]:
trialsvm2 = study.best_trial.params
modelsvm2=svm.SVC(**trialsvm2)
modelsvm2.fit(X_train, y_train)

SVC(C=0.2480354796593595, degree=2, gamma='auto', kernel='poly')

In [360]:
pred=modelsvm2.predict(X_test)
accuracy=accuracy_score(y_test, pred)
print('Accuracy:' ,accuracy*100)

Accuracy: 56.75675675675676
